# SAS Macros - Learning through coding

Following are the 3 common ways to create macro variables.

## The %LET command

%LET command allows for the creation of SAS variables. I have gone through it in P1. This time here is a more practical example.

In [1]:
%let currYear = %sysfunc(year(%sysfunc(today())));
%let nextYear = %eval(&curryear + 1);
%put The value of current year = &currYear;
%put The value of next year = &nextYear;

SAS Connection established. Subprocess id is 10440



## The SYMPUTX function

Another way to create the macro variables is to use the datastep and call the symputx function.<br>The symputx function is used in the SAS datastep programming - only. Following code snippet shows the symputx function in its simplest form.

In [2]:
data _null_;
    cur_yr = year(today());
    lst_yr = cur_yr - 1;
    call symputx("prevYear", lst_yr);
run;

%put The value of last year = &prevYear;

Here is another example of symputx. Note that this example is bit complex. So I will break it down after the code snippet.

In [3]:
data _null_;
    set sashelp.class end=last;
    call symputx(cat("Student", _N_), name);
    if last = 1 then call symputx("Count", _N_);
run;
%macro doloop;
%do i = 1 %to &Count;
    %put &i = &&Student&i;
%end;
%mend;

%doloop;

Here is a little explanation of the previous code snippet.<br>
1. Using the <tag>&#95;</tag>null<tag>&#95;</tag> identifier, I am not creating a dataset but still I am using the full datastep processing power.
2. Setting the flag of end = last helps me to check on reading the last observation in the sashelp.class dataset.
3. The first symputx call creates a macro variable by concatenating a constant string with a counter <tag>&#95;</tag>N<tag>&#95;</tag>. In that variable for each <tag>&#95;</tag>N<tag>&#95;</tag>, I am storing the attribute "name" from sashelp.class.
4. The final symputx stores the total number of records in sashelp.class in a variable, Count.
5. To show that multiple macro variables were created in the first symputx, I created another macro the doloop. Note the use of && in the body of the loop.

## Using INTO clause in PROC SQL

In [4]:
PROC SQL noprint;
SELECT MAX(age) INTO :maxage FROM sashelp.class;
QUIT;

%put Maximum age for any student in the dataset is &maxage;

Note that in PROC SQL, the syntax of creating a macro variable is ":<macro variable name>". Also note that in the log the maximum age is displayed but there are preceeding spaces to the age. In the next piece of code, I am creating two sperate variables and formatting them.

In [5]:
PROC SQL noprint;
SELECT 
        put(MAX(age), 2.)
        , put(MIN(age), 2.)
        INTO :maxage, :minage
FROM sashelp.class;
QUIT;

%put Maximum age for any student in the dataset is &maxage and the minimum age is &minage;

The put function convereted the number into text and stored it into a macro variable.<br>The INTO clause also allows for the creation of macro variables which store multiple values.

In [6]:
PROC SQL noprint;
SELECT DISTINCT make INTO :carmakes SEPARATED BY ", " FROM sashelp.cars;
QUIT;

%put &carmakes;

If I want to extract the macro variable that has a list of values, I can do that too. The following piece of code exactly does that.

In [7]:
data want;
str = "&carmakes";
do i=1 to countw(str,", ");
   value = scan(str,i,", ");
   output;
end;

/*drop other variables if you want*/
drop str i;
run;

proc print data = want (obs=5);
title "Showing first 5 makes";
run;

Obs,value
1,Acura
2,Audi
3,BMW
4,Buick
5,Cadillac


Happy learnng SAS. You will love it.